In [82]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, split, array_join, expr, lower
!pip install numpy
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
! pip install scikit-learn
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from transformers import DistilBertTokenizer
import torch
from sklearn.model_selection import train_test_split


spark = SparkSession.builder \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .appName("BERTDataPreprocessing") \
    .getOrCreate()





In [83]:
#df_comments_MB = spark.read.csv("./mbti9k_comments.csv")  # prob wont use this 
#df_post = spark.read.csv("./typed_posts.csv") # this has numeric data which is not suitable for distilled bert, might try it with differenr model if time allows

# Renaming columns in dataset mbti_1.csv
df_mbti_1 = spark.read.csv("mbti_1.csv", header=True, inferSchema=True)

# Renaming columns in dataset typed_comments.csv
df_comments_typed = spark.read.csv("typed_comments.csv", header=True, inferSchema=True)

from pyspark.sql.functions import col, countDistinct, trim, lower

# Step 1: Normalize 'lang' column to lowercase, trim spaces, and filter only English rows
df_english = df_comments_typed.filter(lower(trim(col("lang"))) == "en")
print(df_english.count())

# Step 3: Count unique users based on the 'author' column
unique_users_count = df_english.select(countDistinct("author")).collect()[0][0]

# Display the number of unique users
print(f"Number of unique English-language users in the Reddit comments dataset: {unique_users_count}")



20492509


[Stage 529:=====================================================> (48 + 1) / 49]

Number of unique English-language users in the Reddit comments dataset: 13616


In [84]:
# Dropping irrelevant columns 

df_comments_typed = (
    df_comments_typed
    .filter(col("comment").isNotNull() & (col("comment") != ""))  # Non-empty comments
    .filter(col("word_count") > 5)  # Minimum word count
)
df_comments_typed_dropped = df_comments_typed[['type', 'comment']]

print(df_comments_typed_dropped.schema)

StructType([StructField('type', StringType(), True), StructField('comment', StringType(), True)])


In [85]:
from pyspark.sql.functions import split, explode, regexp_replace, trim, size, col

# Step 1: Normalize multiple delimiters
df_cleaned = df_mbti_1.withColumn("posts", regexp_replace(df_mbti_1["posts"], r"(\|\|\|)+", "|||"))
print(df_cleaned.count())
# Step 2: Split posts into an array
df_split = df_cleaned.withColumn("post", split(df_cleaned["posts"], r"\|\|\|"))

# Step 3: Explode the array to create separate rows
df_exploded = df_split.select("type", explode(df_split["post"]).alias("post"))

# Step 4: Remove URLs using regex
df_no_links = df_exploded.withColumn("post", regexp_replace(col("post"), r"http\S+|www\S+|\S+\.(com|net|org|io|gov|edu)\S*", ""))

# Step 5: Trim whitespace and remove empty rows
df_trimmed = df_no_links.withColumn("post", trim(col("post"))) \
                        .filter(col("post") != "")

# Step 6: Remove posts with fewer than 5 words
df_mbti_1= df_trimmed.filter(size(split(col("post"), " ")) >= 5)

# Step 7: Show cleaned dataset
df_mbti_1.show(truncate=False)
print(df_mbti_1.count())

# Optional: Save the cleaned dataset
# df_filtered.write.csv("mbti_1_filtered.csv", header=True)


8675
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|type|post                                                                                                                                                                                                     |
+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|INFJ|enfp and intj moments    sportscenter not top ten plays    pranks                                                                                                                                        |
|INFJ|What has been the most life-changing experience in your life?                                                                                            

[Stage 539:>                                                      (0 + 15) / 15]

383864


In [86]:
# Rename 'comment' to 'posts'
df_comments_typed_dropped = df_comments_typed_dropped.withColumnRenamed('comment', 'post')



# Convert all values in the 'type' column to lowercase
from pyspark.sql.functions import lower, col
df_comments_typed_dropped = df_comments_typed_dropped.withColumn("type", lower(col("type")))
df_mbti_1 = df_mbti_1.withColumn("type", lower(col("type")))

#filter out coment less < 5 words 
df_comments_typed_dropped= df_comments_typed_dropped.filter(size(split(col("post"), " ")) >= 5)


# Ensure the schema is exactly the same on both dataframes before joining them
print("Comments_types schema", df_comments_typed_dropped.schema)
print("MBTI schema", df_mbti_1.schema)

Comments_types schema StructType([StructField('type', StringType(), True), StructField('post', StringType(), True)])
MBTI schema StructType([StructField('type', StringType(), True), StructField('post', StringType(), False)])


In [87]:
# Join dataframes
print("Comments_types row count", df_comments_typed_dropped.count())
print("MBTI row count", df_mbti_1.count())

#df_mbti_1.sort(asc("type")).limit(10).show()
#df_comments_typed_dropped.limit(10).show()

df_short_mbti = df_mbti_1.limit(5)
df_short_comments = df_comments_typed_dropped.limit(5)

df_short_union = df_short_mbti.union(df_short_comments)

df_short_union.show()

df_union = df_mbti_1.union(df_comments_typed_dropped)

print("Union dataframe row count", df_union.count())


Comments_types row count 12691384


MBTI row count 383864


+----+--------------------+
|type|                post|
+----+--------------------+
|infj|enfp and intj mom...|
|infj|What has been the...|
|infj|On repeat for mos...|
|infj|May the PerC Expe...|
|infj|The last thing my...|
|entp|Those stats come ...|
|entp|"That's great to ...|
|entp|I can totally agr...|
|entp|"I took it severa...|
|entp|Gawd it's like we...|
+----+--------------------+



[Stage 553:============================================>         (53 + 11) / 64]

Union dataframe row count 13075248


In [88]:
from pyspark.sql.functions import desc, asc
df_union = df_union.sort(asc("type"))


In [89]:
import os

# Define output directory for CSV files
output_dir = "mbti_split_data"
os.makedirs(output_dir, exist_ok=True)  # Ensure directory exists

# Get unique personality types
personality_types = [row[0] for row in df_union.select("type").distinct().collect()]

# Get total number of rows in the dataset
total_rows = df_union.count()

# Dictionary to store row counts and percentages
row_counts = {}

# Iterate over personality types, save CSVs, and count rows
for p_type in personality_types:
    # Filter DataFrame for the current personality type
    df_filtered = df_union.filter(df_union["type"] == p_type)

    # Save the filtered DataFrame as a CSV file with overwrite mode
    file_path = f"{output_dir}/{p_type}.csv"
    df_filtered.write.mode("overwrite").csv(file_path, header=True)

    # Count the number of rows in the filtered DataFrame
    count = df_filtered.count()
    
    # Calculate percentage of total dataset
    percentage = (count / total_rows) * 100
    
    # Store the count and percentage
    row_counts[p_type] = (count, round(percentage, 2))

# Print row counts and percentages
print("✅ CSV files successfully created with the following row counts and percentages:")
for p_type, (count, percentage) in row_counts.items():
    print(f"{p_type}: {count} rows ({percentage}%)")


[Stage 682:======================================================>(63 + 1) / 64]

✅ CSV files successfully created with the following row counts and percentages:
estp: 176051 rows (1.35%)
entj: 1244567 rows (9.52%)
estj: 185015 rows (1.42%)
enfp: 461949 rows (3.53%)
istj: 769810 rows (5.89%)
enfj: 373265 rows (2.85%)
isfj: 453292 rows (3.47%)
istp: 775190 rows (5.93%)
intp: 2910987 rows (22.26%)
infj: 830725 rows (6.35%)
intj: 2258382 rows (17.27%)
isfp: 323723 rows (2.48%)
entp: 813315 rows (6.22%)
infp: 908018 rows (6.94%)
esfp: 204347 rows (1.56%)
esfj: 386612 rows (2.96%)


In [ ]:
! pip install scikit-learn
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from transformers import DistilBertTokenizer
import torch
from sklearn.model_selection import train_test_split
from pyspark.ml.feature import StringIndexer, OneHotEncoder

from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Drop "type_indexed" and "type_encoded" columns if they already exist
for col_name in ["type_indexed", "type_encoded"]:
    if col_name in df_comments_typed.columns:
        df_comments_typed = df_comments_typed.drop(col_name)

# Apply StringIndexer
indexer = StringIndexer(inputCol="type", outputCol="type_indexed", handleInvalid="keep")
df_comments_typed = indexer.fit(df_comments_typed).transform(df_comments_typed)

# Apply OneHotEncoder
encoder = OneHotEncoder(inputCol="type_indexed", outputCol="type_encoded")
df_comments_typed = encoder.fit(df_comments_typed).transform(df_comments_typed)

# Convert Spark DataFrame to Pandas
df_pandas = df_comments_typed.select("comment", "type_encoded").limit(5000).toPandas()
df_pandas["type_encoded"] = df_pandas["type_encoded"].apply(lambda x: x.toArray())
df_pandas["class_label"] = df_pandas["type_encoded"].apply(lambda x: x.argmax())

# Train-Test Split
train_df, test_df = train_test_split(df_pandas, test_size=0.1, stratify=df_pandas["class_label"], random_state=42)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize_text(df): 
    return tokenizer(df["comment"].tolist(), truncation=True, padding=True, max_length=128, return_tensors="pt")

train_encodings, test_encodings = map(tokenize_text, [train_df, test_df])
import numpy as np
import torch

# Convert Pandas arrays to proper NumPy float32 before creating Torch tensors
train_labels, test_labels = map(
    lambda df: torch.tensor(np.stack(df["type_encoded"].values).astype(np.float32)), 
    [train_df, test_df]
)

# Save datasets
for name, enc, lbl in [("train", train_encodings, train_labels), ("test", test_encodings, test_labels)]:
    torch.save({"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"], "labels": lbl}, f"{name}_data.pth", _use_new_zipfile_serialization=False)

print("✅ Datasets saved as train_data.pth & test_data.pth")


In [ ]:
# introvert extrivert separation 